<a href="https://colab.research.google.com/github/vvnjin/community-wellness-dashboard/blob/main/export_community_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This notebook is for exporting all education metrics calculated in distance.ipynb and school_quality.ipynb. Data is exported to an Excel workbook in the  Community Wellness Dashboard - Dfg Google Drive.

The data is formatted to fit a data model, which is read by Tableau to create the community wellness dashboard


Author: Vivian Jin

## Setup GDrive access


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import os

drive_path = '/content/gdrive/MyDrive/Colab Notebooks/Source Data'
drive_contents = os.listdir(drive_path)

# Find the shared drives
shared_drives = [item for item in drive_contents]
print("Shared Drives:", shared_drives)



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Shared Drives: ['01 - Geographic', '02 - Education', '03 - Population']


In [ ]:
import pandas as pd

## 1. Read in existing data, so we can append to existing versions


In [ ]:
LATEST_EDUCATION_METRICS_FILE_NAME = '/content/gdrive/MyDrive/Colab Notebooks/Final Data/results_data_22-Nov-2023.xlsx'

output_wkbk_sheets = pd.read_excel(LATEST_EDUCATION_METRICS_FILE_NAME, sheet_name=None)

print("number of sheets: ", len(output_wkbk_sheets))

for index, sheet in enumerate(output_wkbk_sheets):
  print(sheet)
  globals()[f"{sheet}_df"] = pd.read_excel(LATEST_EDUCATION_METRICS_FILE_NAME, sheet_name=index)


number of sheets:  4
releases
regions
indexes
region_metrics


In [ ]:
releases_df

,id,release_name,release_date
0,0,Rel0.0_22Nov2023,22-Nov-2023


## Create new release version

In [ ]:
from datetime import datetime
# Get the current date and time
current_date = datetime.now().strftime("%d%b%Y")
current_date2 = datetime.now().strftime("%d-%b-%Y")

# SET VERSIONING
RELEASE_ID = releases_df.loc[len(releases_df) - 1, 'id'] + 1
VERSION_NUMBER = 1.0
RELEASE_NAME = f'Rel{VERSION_NUMBER}_{current_date}'
RELEASE_DATE = current_date2

new_row = {'id': RELEASE_ID, 'release_name': RELEASE_NAME, 'release_date': RELEASE_DATE}
releases_df = releases_df.append(new_row, ignore_index=True)

releases_df

<ipython-input-5-febb5e1f8ccb>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  releases_df = releases_df.append(new_row, ignore_index=True)


,id,release_name,release_date
0,0,Rel0.0_22Nov2023,22-Nov-2023
1,1,Rel1.0_08Dec2023,08-Dec-2023


In [ ]:
regions_df

,id,region_name,region_population,region_type,notes
0,0,Vancouver-Fraserview,58686,community,electoral_area
1,1,Vancouver-Hastings,58095,community,electoral_area
2,2,Vancouver-Kensington,58593,community,electoral_area
3,3,Vancouver-Langara,55724,community,electoral_area
4,4,Vancouver-Little Mountain,57034,community,electoral_area
5,5,Vancouver-Point Grey,56818,community,electoral_area
6,6,Vancouver-Quilchena,56957,community,electoral_area
7,7,Vancouver-Renfrew,56922,community,electoral_area
8,8,Vancouver-South Granville,56134,community,electoral_area
9,9,Vancouver-Strathcona,54588,community,electoral_area


In [ ]:
indexes_df

,id,index_name
0,0,Education
1,1,Health
2,2,Housing


# Output the following metrics:
 - median distance for each electoral area for each facility type
 - median distance score for each electoral area
 - mean distance score for vancouver
 - mean capacity percentages for each electoral area
 - mean capacity score for each electoral area (for capacity, we want outliers to impact the scoring)
 - mean capacity score for vancouver
 - median fraser score for each electoral area
 - median fraser score for vancouver

- overall education score for vancouver
- overall education score for each electoral area



In [ ]:
properties_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Final Data/properties_with_distances.csv')


In [ ]:
def aggregate_and_rename(properties_df, agg_type):
    agg_columns = ['closest_library', 'closest_school', 'closest_strongstart_school', 'closest_adult_education_school', 'closest_postsecondary_school']

    results = []

    for col in agg_columns:
        name_col = f"{col}_name"
        dist_col = f"{col}_dist_km"

        if agg_type == 'max':
            agg_df = properties_df.groupby("electoral_area")[name_col].unique().agg(list).reset_index()
            dist_df = properties_df.groupby("electoral_area")[dist_col].max().reset_index()
        elif agg_type == 'mean':
            agg_df = properties_df.groupby("electoral_area")[name_col].unique().agg(list).reset_index()
            dist_df = properties_df.groupby("electoral_area")[dist_col].mean().reset_index()
        elif agg_type == 'median':
            agg_df = properties_df.groupby("electoral_area")[name_col].unique().agg(list).reset_index()
            dist_df = properties_df.groupby("electoral_area")[dist_col].median().reset_index()

        combined_df = pd.merge(agg_df, dist_df, on='electoral_area', how='outer')
        results.append(combined_df)

    cleaned_dfs = []

    for df in results:
        for col in df.columns:
            new_col_name = f"{col}_{agg_type}" if col.endswith('km') else f"{col}_agg_list" if col.endswith('name') else col
            df = df.rename(columns={col: new_col_name})

        cleaned_dfs.append(df)

    final_df = cleaned_dfs[0]

    for i in range(1, len(cleaned_dfs)):
        final_df = final_df.merge(cleaned_dfs[i], on='electoral_area')

    return final_df

In [ ]:
headers =["release_id_fk","region_id_fk","index_id_fk","calculation_type","metric_key","metric_entity","metric_value","metric_unit"]

distances_df = pd.DataFrame(columns=headers)

# VARIABLES TO SET MANUALLY
AGGREGATE = 'median'
INDEX = 'Education'
CALCULATION_TYPE = "distance"
metric_unit = 'km'

# Get variables
release_id_fk = RELEASE_ID
metric_key = f'closest_facility_{AGGREGATE}_distance'
index_id_fk = indexes_df[indexes_df['index_name'] == INDEX]['id'][0]

# Calculate aggregate
median_result = aggregate_and_rename(properties_df, AGGREGATE)

# Melt aggregate df
value_cols = []
id_cols = []
for col in median_result.columns:
  if "km" in col:
    value_cols.append(col)
  elif 'area' in col:
    id_cols.append(col)

melted_df = pd.melt(median_result, id_vars=id_cols, value_vars=value_cols)

for index, row in melted_df.iterrows():
  region_id_fk = regions_df[regions_df['region_name'] == row['electoral_area']]['id']
  metric_entity = row['variable'].split('_')[1]
  metric_value = row['value']

  new_row = {"release_id_fk":release_id_fk, "region_id_fk": region_id_fk, "index_id_fk": index_id_fk, "calculation_type": CALCULATION_TYPE, "metric_key": metric_key, "metric_entity": metric_entity, "metric_value": metric_value, "metric_unit": metric_unit}

  distances_df = pd.concat([distances_df, pd.DataFrame(new_row)], ignore_index=True)

distances_df.sort_values(by='region_id_fk')

,release_id_fk,region_id_fk,index_id_fk,calculation_type,metric_key,metric_entity,metric_value,metric_unit
0,1,0,0,distance,closest_facility_median_distance,library,0.795365,km
24,1,0,0,distance,closest_facility_median_distance,strongstart,0.611747,km
36,1,0,0,distance,closest_facility_median_distance,adult,2.724741,km
12,1,0,0,distance,closest_facility_median_distance,school,0.351449,km
48,1,0,0,distance,closest_facility_median_distance,postsecondary,3.517937,km
37,1,1,0,distance,closest_facility_median_distance,adult,2.794882,km
25,1,1,0,distance,closest_facility_median_distance,strongstart,1.052886,km
49,1,1,0,distance,closest_facility_median_distance,postsecondary,1.599343,km
13,1,1,0,distance,closest_facility_median_distance,school,0.299177,km
1,1,1,0,distance,closest_facility_median_distance,library,1.136348,km


In [ ]:
distance_scores_file = "/content/gdrive/MyDrive/Colab Notebooks/Final Data/electoral_area_with_distance_scores.csv"
distance_scores_file_df = pd.read_csv(distance_scores_file)
distance_scores_file_df

,Unnamed: 0,electoral_area,metric_key,distance_score
0,0,Vancouver-Fraserview,median_distance_score,7.951367
1,1,Vancouver-Hastings,median_distance_score,7.999958
2,2,Vancouver-Kensington,median_distance_score,8.251642
3,3,Vancouver-Langara,median_distance_score,7.877873
4,4,Vancouver-Little Mountain,median_distance_score,8.373820
5,5,Vancouver-Point Grey,median_distance_score,6.653261
6,6,Vancouver-Quilchena,median_distance_score,6.660691
7,7,Vancouver-Renfrew,median_distance_score,8.119015
8,8,Vancouver-South Granville,median_distance_score,7.288850
9,9,Vancouver-Strathcona,median_distance_score,9.039126


In [ ]:

distance_scores_df = pd.DataFrame()
for index, row in distance_scores_file_df.iterrows():
  region_id_fk = regions_df[regions_df['region_name'] == row['electoral_area']]['id']
  metric_entity = "all_education_facilities"
  metric_value = row['distance_score']
  if row['electoral_area'] == 'Vancouver':
    metric_key = 'city_mean_distance_score'

  else:
    metric_key = 'median_distance_score'
  metric_unit = "pts"

  new_row = {"release_id_fk":release_id_fk, "region_id_fk": region_id_fk, "index_id_fk": index_id_fk, "calculation_type": CALCULATION_TYPE, "metric_key": metric_key, "metric_entity": metric_entity, "metric_value": metric_value, "metric_unit": metric_unit}
  distance_scores_df = pd.concat([distance_scores_df, pd.DataFrame(new_row)], axis=0)

distance_scores_df

,release_id_fk,region_id_fk,index_id_fk,calculation_type,metric_key,metric_entity,metric_value,metric_unit
0,1,0,0,distance,median_distance_score,all_education_facilities,7.951367,pts
1,1,1,0,distance,median_distance_score,all_education_facilities,7.999958,pts
2,1,2,0,distance,median_distance_score,all_education_facilities,8.251642,pts
3,1,3,0,distance,median_distance_score,all_education_facilities,7.877873,pts
4,1,4,0,distance,median_distance_score,all_education_facilities,8.373820,pts
5,1,5,0,distance,median_distance_score,all_education_facilities,6.653261,pts
6,1,6,0,distance,median_distance_score,all_education_facilities,6.660691,pts
7,1,7,0,distance,median_distance_score,all_education_facilities,8.119015,pts
8,1,8,0,distance,median_distance_score,all_education_facilities,7.288850,pts
9,1,9,0,distance,median_distance_score,all_education_facilities,9.039126,pts


## Merge distances and distance scores dfs

In [ ]:
merged_distance_scores_df = pd.concat([distances_df, distance_scores_df], ignore_index=True)
merged_distance_scores_df.head()

,release_id_fk,region_id_fk,index_id_fk,calculation_type,metric_key,metric_entity,metric_value,metric_unit
0,1,0,0,distance,closest_facility_median_distance,library,0.795365,km
1,1,1,0,distance,closest_facility_median_distance,library,1.136348,km
2,1,2,0,distance,closest_facility_median_distance,library,0.933617,km
3,1,3,0,distance,closest_facility_median_distance,library,1.193681,km
4,1,4,0,distance,closest_facility_median_distance,library,0.869728,km


## Calculate mean capacity percentages per electoral area and for Vancouver

In [ ]:
school_qualities_file = "/content/gdrive/MyDrive/Colab Notebooks/Intermediate Data/02 - Education/school_quality_fraser_and_capacity_scores.csv"
school_qualities_df = pd.read_csv(school_qualities_file)

mean_capacity_df = pd.DataFrame(school_qualities_df.groupby('electoral_area')['school_capacity_%'].mean().reset_index())
mean_capacity_df = mean_capacity_df.rename(columns={'school_capacity_%':'metric_value'})
mean_capacity_df['metric_key'] = 'mean_capacity_%'
mean_capacity_df['metric_unit'] = 'percentage'

total_capacity_score = mean_capacity_df['metric_value'].mean()

vancouver_row = "Vancouver", total_capacity_score,'city_mean_capacity_%' , 'percentage'
mean_capacity_df.loc[len(mean_capacity_df)] = vancouver_row

mean_capacity_df

,electoral_area,metric_value,metric_key,metric_unit
0,Vancouver-Fraserview,84.923077,mean_capacity_%,percentage
1,Vancouver-Hastings,83.069326,mean_capacity_%,percentage
2,Vancouver-Kensington,75.909091,mean_capacity_%,percentage
3,Vancouver-Langara,94.736364,mean_capacity_%,percentage
4,Vancouver-Little Mountain,103.357143,mean_capacity_%,percentage
5,Vancouver-Point Grey,95.051956,mean_capacity_%,percentage
6,Vancouver-Quilchena,94.738889,mean_capacity_%,percentage
7,Vancouver-Renfrew,86.938836,mean_capacity_%,percentage
8,Vancouver-South Granville,105.368127,mean_capacity_%,percentage
9,Vancouver-Strathcona,83.694215,mean_capacity_%,percentage


In [ ]:
quality_scores_df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/Final Data/electoral_area_with_fraser_and_capacity_scores.csv")

# Calculate Vancouver totals
total_fraser_score = quality_scores_df['median_Fraser_score'].mean()
total_capacity_score = quality_scores_df['mean_capacity_score'].mean()

quality_scores_df = quality_scores_df.melt('electoral_area').rename(columns={'variable':'metric_key', 'value':'metric_value'})
quality_scores_df

vancouver_fraser_score_row = 'Vancouver', 'city_mean_Fraser_score', total_fraser_score
quality_scores_df.loc[len(quality_scores_df)] = vancouver_fraser_score_row

vancouver_capacity_score_row = 'Vancouver', 'city_mean_capacity_score', total_capacity_score
quality_scores_df.loc[len(quality_scores_df)] = vancouver_capacity_score_row

quality_scores_df['metric_unit'] = 'pts'
quality_scores_df

,electoral_area,metric_key,metric_value,metric_unit
0,Vancouver-Fraserview,median_Fraser_score,6.600000,pts
1,Vancouver-Hastings,median_Fraser_score,6.600000,pts
2,Vancouver-Kensington,median_Fraser_score,5.350000,pts
3,Vancouver-Langara,median_Fraser_score,6.400000,pts
4,Vancouver-Little Mountain,median_Fraser_score,7.350000,pts
5,Vancouver-Point Grey,median_Fraser_score,7.700000,pts
6,Vancouver-Quilchena,median_Fraser_score,7.300000,pts
7,Vancouver-Renfrew,median_Fraser_score,8.250000,pts
8,Vancouver-South Granville,median_Fraser_score,7.600000,pts
9,Vancouver-Strathcona,median_Fraser_score,6.500000,pts


## Merge capacity_% and mean_capacity_%, and median_Fraser_score dfs

In [ ]:
merged_quality_scores_df = pd.concat([quality_scores_df, mean_capacity_df], ignore_index=True)
merged_quality_scores_df.head()

,electoral_area,metric_key,metric_value,metric_unit
0,Vancouver-Fraserview,median_Fraser_score,6.60,pts
1,Vancouver-Hastings,median_Fraser_score,6.60,pts
2,Vancouver-Kensington,median_Fraser_score,5.35,pts
3,Vancouver-Langara,median_Fraser_score,6.40,pts
4,Vancouver-Little Mountain,median_Fraser_score,7.35,pts


In [ ]:
merged_quality_scores_df
CALCULATION_TYPE = 'quality'

complete_quality_scores_df = pd.DataFrame()
for index, row in merged_quality_scores_df.iterrows():
  region_id_fk = regions_df[regions_df['region_name'] == row['electoral_area']]['id']
  metric_entity = 'schools'
  metric_key = row['metric_key']
  metric_value = row['metric_value']
  metric_unit = row['metric_unit']

  new_row = {"release_id_fk":release_id_fk, "region_id_fk": region_id_fk, "index_id_fk": index_id_fk, "calculation_type": CALCULATION_TYPE, "metric_key": metric_key, "metric_entity": metric_entity, "metric_value": metric_value, "metric_unit": metric_unit}
  complete_quality_scores_df = pd.concat([complete_quality_scores_df, pd.DataFrame(new_row)], axis=0)

complete_quality_scores_df

,release_id_fk,region_id_fk,index_id_fk,calculation_type,metric_key,metric_entity,metric_value,metric_unit
0,1,0,0,quality,median_Fraser_score,schools,6.600000,pts
1,1,1,0,quality,median_Fraser_score,schools,6.600000,pts
2,1,2,0,quality,median_Fraser_score,schools,5.350000,pts
3,1,3,0,quality,median_Fraser_score,schools,6.400000,pts
4,1,4,0,quality,median_Fraser_score,schools,7.350000,pts
5,1,5,0,quality,median_Fraser_score,schools,7.700000,pts
6,1,6,0,quality,median_Fraser_score,schools,7.300000,pts
7,1,7,0,quality,median_Fraser_score,schools,8.250000,pts
8,1,8,0,quality,median_Fraser_score,schools,7.600000,pts
9,1,9,0,quality,median_Fraser_score,schools,6.500000,pts


## Merge all school distance and quality metrics

In [ ]:
scores_df = pd.concat([complete_quality_scores_df, merged_distance_scores_df], ignore_index =True)
scores_df.sort_values(by=['region_id_fk', 'metric_key'])

,release_id_fk,region_id_fk,index_id_fk,calculation_type,metric_key,metric_entity,metric_value,metric_unit
39,1,0,0,distance,closest_facility_median_distance,library,0.795365,km
51,1,0,0,distance,closest_facility_median_distance,school,0.351449,km
63,1,0,0,distance,closest_facility_median_distance,strongstart,0.611747,km
75,1,0,0,distance,closest_facility_median_distance,adult,2.724741,km
87,1,0,0,distance,closest_facility_median_distance,postsecondary,3.517937,km
...,...,...,...,...,...,...,...,...
110,1,11,0,distance,median_distance_score,all_education_facilities,8.387029,pts
24,1,12,0,quality,city_mean_Fraser_score,schools,6.720833,pts
38,1,12,0,quality,city_mean_capacity_%,schools,95.162807,percentage
25,1,12,0,quality,city_mean_capacity_score,schools,8.302934,pts


In [ ]:
electoral_area_education_scores

,region_id_fk,metric_value
0,0,7.718559
1,1,7.667181
2,2,7.209244
3,3,7.767583
4,4,7.894073
5,5,7.831187
6,6,7.489114
7,7,8.330267
8,8,7.735138
9,9,7.697098


## Calculate overall Vancouver education score, and overall education scores for each electoral area


In [ ]:
all_scores_df = scores_df

electoral_area_education_scores = all_scores_df[(~all_scores_df['metric_key'].str.startswith('city')) & (all_scores_df['metric_unit'] == 'pts')]
electoral_area_education_scores = pd.DataFrame(electoral_area_education_scores.groupby("region_id_fk")['metric_value'].mean().reset_index())

for index, row in electoral_area_education_scores.iterrows():
    new_row = {"release_id_fk":release_id_fk, "region_id_fk": int(row['region_id_fk']), "index_id_fk": index_id_fk, "calculation_type": 'overall_education', "metric_key": 'overall_education_score', "metric_entity": 'distance_and_school_quality', "metric_value": row['metric_value'], "metric_unit": 'pts'}
    all_scores_df = all_scores_df.append(new_row, ignore_index=True)

vancouver_id_fk = int(regions_df[regions_df['region_name'] == 'Vancouver']['id'])
vancouver_education_score = scores_df[(scores_df['metric_key'].str.startswith('city')) & (all_scores_df['metric_unit'] == 'pts')]['metric_value'].mean()
vancouver_row = {"release_id_fk":release_id_fk, "region_id_fk": vancouver_id_fk, "index_id_fk": index_id_fk, "calculation_type": 'overall_education', "metric_key": 'city_overall_education_score', "metric_entity": 'distance_and_school_quality', "metric_value": vancouver_education_score, "metric_unit": 'pts'}
all_scores_df = all_scores_df.append(vancouver_row, ignore_index=True)

all_scores_df

<ipython-input-70-e0051d56db48>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_scores_df = all_scores_df.append(new_row, ignore_index=True)
<ipython-input-70-e0051d56db48>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_scores_df = all_scores_df.append(new_row, ignore_index=True)
<ipython-input-70-e0051d56db48>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_scores_df = all_scores_df.append(new_row, ignore_index=True)
<ipython-input-70-e0051d56db48>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_scores_df = all_scores_df.append(new_row, ignore_index=True)
<ipython-input-70-e0051d56db48>:8: FutureWarning: The frame.

,release_id_fk,region_id_fk,index_id_fk,calculation_type,metric_key,metric_entity,metric_value,metric_unit
0,1,0,0,quality,median_Fraser_score,schools,6.600000,pts
1,1,1,0,quality,median_Fraser_score,schools,6.600000,pts
2,1,2,0,quality,median_Fraser_score,schools,5.350000,pts
3,1,3,0,quality,median_Fraser_score,schools,6.400000,pts
4,1,4,0,quality,median_Fraser_score,schools,7.350000,pts
...,...,...,...,...,...,...,...,...
120,1,8,0,overall_education,overall_education_score,distance_and_school_quality,7.735138,pts
121,1,9,0,overall_education,overall_education_score,distance_and_school_quality,7.697098,pts
122,1,10,0,overall_education,overall_education_score,distance_and_school_quality,6.187936,pts
123,1,11,0,overall_education,overall_education_score,distance_and_school_quality,8.009169,pts


## Output to Excel workbook

In [ ]:
current_date = datetime.now().strftime("%d-%b-%Y")

output_file_name = f'/content/gdrive/MyDrive/Colab Notebooks/Final Data/education_metrics_{current_date}.xlsx'

with pd.ExcelWriter(output_file_name) as writer:
    # Write each dataframe to a different sheet
    releases_df.to_excel(writer, sheet_name='releases', index=False)
    regions_df.to_excel(writer, sheet_name='regions', index=False)
    indexes_df.to_excel(writer, sheet_name='indexes', index=False)
    all_scores_df.to_excel(writer, sheet_name='region_metrics', index=False)